# Content
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

[2. Introduction](#Introduction)

[2.1 Overview](#Overview)

[2.2 Importing the important libraries](#Importing-the-important-libraries)

[2.3 FastAPI Application Initialization](#FastAPI-Application-Initialization)

[2.4 FastAPI Endpoint Description](#FastAPI-Endpoint-Description)

[2.5 FastAPI Endpoint Description](#FastAPI-Endpoint-Description)

# Introduction
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

This notebook serves as a guide to using FastAPI to create endpoints for a frontend or dashboard application. The primary goal is to enable interaction with a machine learning model for predictive tasks. Specifically, the notebook demonstrates the setup and utilization of FastAPI endpoints for processing and prediction.

## Overview

- FastAPI, a modern web framework for building APIs, plays a pivotal role in this process. It facilitates the creation of endpoints to receive and respond to HTTP requests effectively.

- The primary objective of this notebook is to illustrate the implementation of an endpoint that allows users to upload files or data for processing. In the context of machine learning, this endpoint can be used to make predictions, conduct analysis, and provide responses to client applications.

- The notebook delves into the code required for creating and configuring the FastAPI application, defining endpoints, processing uploaded data, and returning results. The main endpoint described in this notebook expects file uploads, processes the data, and saves the processed results in a structured manner.

- Throughout the notebook, you will find explanations of the code, including the use of FastAPI decorators and middleware, data processing steps, and error handling strategies.

- The endpoint presented here is just one example of what FastAPI can achieve. It can serve as a foundation for building more complex and customized APIs tailored to specific use cases.

## Importing the important libraries
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [2]:
from pydantic import BaseModel
from typing import Optional
from fastapi.responses import JSONResponse, FileResponse
from fastapi import FastAPI, File, UploadFile
from app.model.model import predict_pipeline
import numpy as np
import pandas as pd
import cv2
import io
from fastapi.middleware.cors import CORSMiddleware
import json
import os
import shutil
import uvicorn


## FastAPI Application Initialization
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

1. FastAPI Application with CORS Configuration

2. Create a FastAPI application instance.
    
    **app = FastAPI()**
        
- Define a list of allowed origins for Cross-Origin Resource Sharing (CORS).
   
   **orings = ["*"]**

- Add CORS middleware to the FastAPI application.

3. app.add_middleware( CORSMiddleware,

- Allow requests from any origin ("*")

    **allow_origins=orings**   
      
- Allow including credentials (e.g., cookies) with requests 

    **allow_credentials=True**
    
- Allow all HTTP methods

    **allow_methods=["*"]** 
    
- Allow all headers in requests.

    **allow_headers=["*"]**       



In [3]:
app = FastAPI()

orings = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins = orings,
    allow_credentials = True,
    allow_methods = ["*"],
    allow_headers = ["*"]
)

## FastAPI Endpoint Description
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

- In this section, we define a FastAPI endpoint that responds to `HTTP GET` requests to the root `("/")` path. The endpoint provides a simple health check response.


In [4]:
@app.get("/")
def home():
    return {"health_check": "Sandy is Here"}

## FastAPI Endpoint Description
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
In this section, we define a FastAPI endpoint for file upload and processing. The endpoint expects a file upload, performs processing using a loaded model, and saves the results in `JSON format`.


 **1. Request Model Definition**
- We define a request model `PredictionRequest` using FastAPI's `BaseModel`. It specifies that the endpoint expects a file upload.

 **2. Endpoint Definition**
 - We define an endpoint using the `@app.post("/predict/")` decorator, indicating that this endpoint responds to `HTTP POST` requests at the `"/predict/"` path. It receives a file upload.
 
 **3. File Processing**
- The code within the endpoint processes the uploaded file. It reads the content of the file, converts it to a CSV format, and performs processing using the predict_pipeline function.

**4. Data Saving**
- The results of the processing are saved as JSON files. Each JSON file corresponds to a specific faulty index and is organized in folders within the base directory.

**5. Error Handling**
- The code includes error handling to catch any exceptions and provide appropriate responses.

- The endpoint returns a JSON response with a success message, drift data, and information about faulty indexes upon successful execution.

- In case of an error, the endpoint provides an error message with a status code of 500.

- This endpoint is designed to handle file uploads, process the data, and save the results in a structured manner for further analysis.

In [5]:
class PredictionRequest(BaseModel):
    file: UploadFile


# Define the base directory where prediction folders will be created
base_directory = "faulty_data"

@app.post("/predict/")
async def predict(file: UploadFile):
    try:
        file_name = file.filename

        content = await file.read()
        csv_file_name = io.StringIO(content.decode("utf-8"))

        # Perform your processing using the loaded model here
        faulty_indexes, faulty_dic_list, drift_data = predict_pipeline(csv_file_name)

        # Define the base directory for saving data
        base_directory = "/GM_usecase/backend_fastapi/app/faulty_data"

        # Ensure the base directory exists within the container
        os.makedirs(base_directory, exist_ok=True)

        # Clean up the old prediction folders (delete previous data)
        for folder_name in os.listdir(base_directory):
            folder_path = os.path.join(base_directory, folder_name)
            if os.path.isdir(folder_path):
                shutil.rmtree(folder_path)

        # Iterate through each faulty index and create a folder for it
        for faulty_index, (index, data_dict) in zip(faulty_indexes, enumerate(faulty_dic_list)):
            new_folder = os.path.join(base_directory, f"prediction_{faulty_index}")
            os.makedirs(new_folder, exist_ok=True)

            # Save each dictionary in faulty_dic_list as a separate JSON file in the new folder
            filename = os.path.join(new_folder, f"faulty_index_{index}.json")
            with open(filename, "w") as json_file:
                json.dump(data_dict, json_file)

        return JSONResponse(content={"Message": "Data saved as JSON","Drift Data": drift_data,"faulty_index":faulty_indexes}, status_code=201)
    except Exception as e:
        return JSONResponse(content={"message": f"An error occurred: {str(e)}"}, status_code=500)


######################################### END ############################################